In [1]:
import matplotlib.pyplot as plt, rdflib, pandas as pd, numpy as np, sys, os, random, math, fiona, uuid, copy, glob
from osgeo import gdal, osr, gdal_array
from collections import defaultdict, Counter
from dotenv import load_dotenv
from tqdm.auto import tqdm
from typing import *
from ruamel.yaml import YAML
import xarray as xr

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# next cell
%reload_ext autoreload

In [2]:
load_dotenv(verbose=True)
paths = ["../", "/workspace/d-repr/pydrepr", "/home/rook/workspace/d-repr/pydrepr"]
for path in paths:
    if path not in sys.path:
        sys.path.insert(0, path)

yaml = YAML()

from drepr import __version__, DRepr, outputs
from drepr.executors.readers.reader_container import ReaderContainer
from drepr.executors.readers.np_dict import NPDictReader
print("drepr version:", __version__)

drepr version: 2.8


In [3]:
from funcs import DcatReadFunc
from funcs.trans_cropping_func import CroppingTransFunc
from funcs.readers.dcat_read_func import ShardedClassID
from funcs.gdal.raster import *
from dateutil.parser import parse

**1. configuration & global variables**

In [4]:
HOME_DIR = os.environ['HOME_DIR']

gldas = "5babae3f-c468-4e01-862e-8b201468e3b5"
gpm = "ea0e86f3-9470-4e7e-a581-df85b4a7075d"
region = "74e6f707-d5e9-4cbd-ae26-16ffa21a1d84"
variable = "atmosphere_water__precipitation_mass_flux"
variable = "land_surface_air__temperature"

ethiopia = BoundingBox(32.75418, 3.22206, 47.98942, 15.15943)

**2. download the weather dataset**

In [5]:
def read_datasets(dataset_id, start_time, end_time):
  if start_time is not None:
    start_time = parse(start_time)
  if end_time is not None:
    end_time = parse(end_time)
    
  func = DcatReadFunc(dataset_id, start_time, end_time)
  func.set_preferences({"data": "array"})
  datasets = func.exec()['data']
  return datasets

def read_local_datasets(repr_file, resource_path):
  drepr = DRepr.parse_from_file(repr_file)
  datasets = []
  for file in glob.glob(resource_path):
    datasets.append(outputs.ArrayBackend.from_drepr(drepr, file))  
  if len(datasets) > 1:
    return ShardedBackend(datasets)
  return datasets[0]

In [ ]:
weather_dataset = read_datasets(gldas, "2016-01-01T00:00:00", "2017-01-01T00:00:00")

2020-03-05 04:31:52,974 | funcs.readers.dcat_read_func | INFO - Overwrite GLDAS
2020-03-05 04:31:52,975 | funcs.readers.dcat_read_func | INFO - Found key 'resource_repr'
2020-03-05 04:31:52,975 | funcs.readers.dcat_read_func | INFO - Downloading 2928 resources ...
2020-03-05 04:31:53,057 | funcs.readers.dcat_read_func | INFO - Download Complete. Skip 2928 and download 0 resources


**3. crop the data**

some useful functions to convert datasets to rasters and convert them back

In [ ]:
with open(HOME_DIR + "/examples/d3m/crop_bb.yml", "r") as f:
  crop_bb_conf = yaml.load(f)

def dataset2raster(sm, variable):
  rasters = []
  for c in sm.c('mint:Variable').filter(outputs.FCondition("mint:standardName", "==", variable)):
    for raster_id, sc in c.group_by("mint-geo:raster"):
      # TODO: handle time properly
      timestamp = sc.p("mint:timestamp").as_ndarray([])
      if timestamp.data.size != 1:
        raise NotImplemented()
      timestamp = timestamp.data[0]
      
      data = sc.p("rdf:value").as_ndarray([sc.p("mint-geo:lat"), sc.p("mint-geo:long")])
      gt_info = sm.get_record_by_id(raster_id)
      gt = GeoTransform(x_0=gt_info.s("mint-geo:x_0"),
                        y_0=gt_info.s("mint-geo:y_0"),
                        dx=gt_info.s("mint-geo:dx"), dy=gt_info.s("mint-geo:dy"))
      raster = Raster(data.data, gt, int(gt_info.s("mint-geo:epsg")),
             float(data.nodata.value) if data.nodata is not None else None)
      raster.timestamp = timestamp
      rasters.append(raster)
  return rasters
  
def raster2dataset(r, variable):
  global crop_bb_conf
  reader = NPDictReader({
    "variable": r.data,
    "lat": r.get_center_latitude(),
    "long": r.get_center_longitude(),
    "timestamp": r.timestamp,
    "standard_name": variable,
    "gt_x_0": r.geotransform.x_0,
    "gt_y_0": r.geotransform.y_0,
    "gt_dx": r.geotransform.dx,
    "gt_dy": r.geotransform.dy,
    "gt_epsg": r.epsg,
    "gt_x_slope": r.geotransform.x_slope,
    "gt_y_slope": r.geotransform.y_slope,
  })
  resource_id = str(uuid.uuid4())
  ReaderContainer.get_instance().set(resource_id, reader)
  
  conf = copy.deepcopy(crop_bb_conf)
  conf['attributes']['variable']['missing_values'].append(r.nodata)
  drepr = DRepr.parse(conf)
  sm = outputs.ArrayBackend.from_drepr(drepr, resource_id)
  ReaderContainer.get_instance().delete(resource_id)
  return sm

def raster2netcdf(r, variable, outfile):
  lat = r.get_center_latitude()
  long = r.get_center_longitude()
  data = xr.DataArray(r.data, dims=('lat', 'long'), coords={'lat': lat, 'long': long})
  data.attrs['standard_name'] = variable
  data.attrs['_FillValue'] = r.nodata
  data.attrs['missing_values'] = r.nodata
  
  ds = xr.Dataset({standard_name: data})  
  ds.to_netcdf(outfile)
  
def dataset2netcdf(sm):
  assert len(sm.c("mint:Variable")) == 1
  c = sm.c("mint:Variable")[0]
  if c.p("mint:Place") is not None:
    raise NotImplemented()
    
  standard_name = c.p("mint:standardName").as_ndarray([]).data
  assert standard_name.size == 1
  standard_name = standard_name[0]

  timestamp = c.p("mint:timestamp").as_ndarray([]).data
  assert timestamp.size == 1
  timestamp = timestamp[0]
  
  groups = list(c.group_by("mint-geo:raster"))
  assert len(groups) == 1
  gt = sm.get_record_by_id(groups[0][0])

  val = c.p("rdf:value").as_ndarray([c.p("mint-geo:lat"), c.p("mint-geo:long")])
  data = val.data.reshape(1, *val.data.shape)
  data = xr.DataArray(val.data.reshape(1, *val.data.shape), dims=('time', 'lat', 'long'), coords={
    'lat': val.index_props[0], 'long': val.index_props[1], 'time': np.asarray([timestamp])
  })
  data.attrs['standard_name'] = standard_name
  data.attrs['_FillValue'] = val.nodata.value
  data.attrs['missing_values'] = val.nodata.value
  
  ds = xr.Dataset({"variable": data})
  ds.attrs.update({
    "conventions": "CF-1.6",
    "dx": gt.s('mint-geo:dx'),
    "dy": gt.s("mint-geo:dy"),
    "epsg": gt.s("mint-geo:epsg"),
    "x_slope": gt.s("mint-geo:x_slope"),
    "y_slope": gt.s("mint-geo:y_slope"),
    "x_0": gt.s("mint-geo:x_0"),
    "y_0": gt.s("mint-geo:y_0")
  })
  return ds

**3.1 crop the data by a bounding box**

In [ ]:
subrasters = []
for raster in tqdm(dataset2raster(weather_dataset, variable)):
  sr = raster.crop(bounds=ethiopia, resampling_algo=ReSample.BILINEAR)
  sr.timestamp = raster.timestamp
  filename = datetime.datetime.utcfromtimestamp(sr.timestamp).strftime("%Y%m%d%H%M%S")
  sm = raster2dataset(sr, variable)
  dataset2netcdf(sm).to_netcdf(HOME_DIR + f"/data/gldas/{variable}/{filename}.nc4")
  subrasters.append(sr)

debug to see if the data is correct

In [32]:
subrasters[-1].to_geotiff(HOME_DIR + "/data/debug/small.tif")
raster.to_geotiff(HOME_DIR + "/data/debug/full.tif")
sm = read_local_datasets(HOME_DIR + "/examples/d3m/gldas.crop.yml", HOME_DIR + f"/data/gldas/{variable}/201109*.nc4")
a = dataset2raster(sm, variable)[0].data
b = subrasters[-1].data

assert np.allclose(a, b)

**3.2 crop data by shapefiles**

In [33]:
def shape_array_to_shapefile(data, fname):
  polygon = data[0]
  if isinstance(polygon[0][0][0], (int, float)):
    shape_type = 'Polygon'
  else:
    shape_type = 'MultiPolygon'

  epsg = fiona.crs.from_epsg(data[1])
  driver = "ESRI Shapefile"
  polygon = {
      'geometry': {
          'type': shape_type,
          'coordinates': polygon
      },
      'properties': {
          'name': 'TempCroppingPolygon'
      }
  }
  schema = {'geometry': shape_type, 'properties': {'name': 'str'}}
  with fiona.open(fname, 'w', crs='+datum=WGS84 +ellps=WGS84 +no_defs +proj=longlat', driver=driver, schema=schema) as shapefile:
    shapefile.write(polygon)
    
def create_shapefile(sm, dname):
  shape_files = []
  for c in sm.c("mint:Place"):
    for r in c.iter_records():
      polygon = sm.get_record_by_id(r.s('mint-geo:bounding')).s('rdf:value')
      shape_file = HOME_DIR + f'/data/{dname}/{r.s("mint:region").replace(" ", "-")}.shp'
      shape_array_to_shapefile([polygon, 4326], shape_file)
      shape_files.append({
        "file": shape_file,
        "region": r.s("mint:region")
      })
  return shape_files

In [34]:
region_dataset = read_datasets(region, None, None)

2020-03-05 03:11:27,490 | funcs.readers.dcat_read_func | INFO - Found key 'dataset_repr'
2020-03-05 03:11:27,491 | funcs.readers.dcat_read_func | INFO - Downloading 1 resources ...
2020-03-05 03:11:27,493 | funcs.readers.dcat_read_func | INFO - Download Complete. Skip 1 and download 0 resources


In [35]:
shape_files = create_shapefile(region_dataset, 'debug/regions')

In [36]:
cropped_dataset = read_local_datasets(HOME_DIR + "/examples/d3m/gldas.crop.yml", 
                                      HOME_DIR + f"/data/gldas/{variable}/201109*.nc4")

In [39]:
region_rasters = []

for raster in tqdm(dataset2raster(cropped_dataset, variable)):
  raster.to_geotiff(HOME_DIR + f"/data/region_full.tif")
  for shape_file in shape_files:
    sr = raster.crop(vector_file=shape_file['file'],
                      resampling_algo=ReSample.BILINEAR,
                      touch_cutline=True)
    region_rasters.append(sr)
    sr.to_geotiff(HOME_DIR + f"/data/region_{shape_file['region'].replace(' ', '-')}.tif")
    print(np.sum(sr.data != -9999))

4
103
501
6
4
195
67
101
251
169
535



-----------

In [257]:
sm = raster2dataset(rasters[0], variable)

In [272]:
dataset2netcdf(sm).to_netcdf(HOME_DIR + "/data/tmp_out/test.nc4")

In [222]:
for raster in rasters:
  raster.to_geotiff(HOME_DIR + "/data/full.tif")
  ethiopia_raster = raster.crop(bounds=ethiopia, resampling_algo=ReSample.BILINEAR)
  ethiopia_raster.to_geotiff(HOME_DIR + "/data/small.tif")

In [223]:
sm = raster2dataset(ethiopia_raster, variable)

In [235]:
dataset2netcdf(sm)

None
defaultdict(<class 'list'>, {'rdf:value': [0], 'mint-geo:lat': [1], 'mint-geo:long': [2], 'mint:standardName': [3], 'mint-geo:raster': [4], 'mint:timestamp': []})
defaultdict(<class 'list'>, {'rdf:value': [0], 'mint-geo:lat': [1], 'mint-geo:long': [2], 'mint:standardName': [3], 'mint-geo:raster': [4], 'mint:timestamp': []})


IndexError: list index out of range